In [ ]:
from transformers import SiglipForImageClassification, AutoImageProcessor
import os
import pandas as pd
from PIL import Image
from datasets import Dataset
from transformers import Trainer, TrainingArguments
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import torch

In [ ]:
! apt-get install -y gdown

In [ ]:
# download zip from google drive and unzip contents
! gdown --id 1rbISVuHbT_AJPw4wv4ywLym3TLVuGLDH      

In [ ]:
# Unzip the file
import zipfile
with zipfile.ZipFile("./frames_and_annotations.zip", "r") as zip_ref:
    zip_ref.extractall("frames")


In [ ]:
df = pd.read_csv("./frames/frames_annotations.csv")

# fix backslash to forward slash in relative paths
df["image"] = df["image"].apply(lambda x: os.path.join("frames", x.replace("\\", "/")))

#assign labels
df["label"] = df["label"].map({"not_taking_medication": 0, "taking_medication": 1})

# generate dataset from df
dataset = Dataset.from_pandas(df)

In [ ]:
model = SiglipForImageClassification.from_pretrained("prithivMLmods/Human-Action-Recognition")
processor = AutoImageProcessor.from_pretrained("prithivMLmods/Human-Action-Recognition")

In [ ]:
def preprocess(example):
    image = Image.open(example["image"]).convert("RGB")
    inputs = processor(images=image, return_tensors="pt")
    inputs["labels"] = example["label"]
    return {
        "pixel_values": inputs["pixel_values"].squeeze(),
        "labels": inputs["labels"]
    }

In [ ]:
dataset = dataset.map(preprocess, remove_columns=["image", "label"])
dataset = dataset.train_test_split(test_size=0.1)

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    # Ensure logits are numpy arrays
    if isinstance(logits, tuple):
        print("logits is tuple")
        logits = logits[0]
    if isinstance(logits, torch.Tensor):
        print("logits is tensor")
        logits = logits.detach().cpu().numpy()
    if isinstance(labels, torch.Tensor):
        print("labels is tensor")
        labels = labels.detach().cpu().numpy()
    preds = np.argmax(logits, axis=1)
    acc = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="binary", zero_division=0)
    return {
        "accuracy": acc,
        "precision": precision,
        "recall": recall,
        "f1": f1,
    }

In [ ]:
model.config.label2id = {"not_taking_medication": 0, "taking_medication": 1}
model.config.id2label = {0: "not_taking_medication", 1: "taking_medication"}

training_args = TrainingArguments(
    output_dir="./HAR-medication-finetuned",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    eval_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=10,
    hub_model_id="Adekiii/HAR-medication-finetuned",
    report_to=["none"],
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    compute_metrics=compute_metrics,
)

In [ ]:
# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

In [ ]:
trainer.train()

In [ ]:
trainer.save_model("./HAR-med-finetunedv2")
processor.save_pretrained("./HAR-med-finetunedv2")

In [ ]:
from huggingface_hub import HfApi

access_token = "..."

api = HfApi(token=access_token)
api.upload_folder(
    folder_path="HAR-med-finetunedv2",
    repo_id="tam6/HAR-medication-finetunedv2", # change with own repo
    repo_type="model",
)

In [ ]:
# Load the processor and model from the HuggingFace Hub
processor = AutoImageProcessor.from_pretrained("Adekiii/HAR-medication-finetuned")
model = SiglipForImageClassification.from_pretrained("Adekiii/HAR-medication-finetuned")

In [ ]:
test_data = dataset["test"]["features"]
print(test_data)

In [ ]:
print(dataset)

In [ ]:
inputs = processor(images=test_data["pixel_values"], return_tensors="pt")

# Run inference
predicted_labels = []
true_labels = test_data["label"]
inputs = test_data["pixel_values"]
with torch.no_grad():
    #for im in test_data["pixel_values"]
    outputs = model(**inputs)
    logits = outputs.logits
    predicted_class_idx = logits.argmax(-1).item()
    #predicted_labels.append(predicted_class_idx)
precision, recall, f1, _ = precision_recall_fscore_support(true_labels, predicted_class_idx, average="binary", zero_division=0)


In [ ]:
print("f1 score on test set: " + str(f1))